# Media-Pipe Preprocessing

## Convert Multiple Raw Videos into Mediapipe Stick-Figures

In [ ]:

import cv2
import mediapipe as mp
import numpy as np
import time
import os
import glob

parent_directory='ndata'
# video_dir='म घर मा धेरै काम गर्छु ।/'


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands


prev_frame_time=0
new_frame_time=0

# For webcam input:
# cap = cv2.VideoCapture(0)
for video_dirs in glob.glob(parent_directory+'/*'):

  video_dir=video_dirs.removeprefix(parent_directory+'/')

  for f in range(2):
    for video in glob.glob(video_dirs+'/*'):

      cap=cv2.VideoCapture(video)

      fourcc = cv2.VideoWriter_fourcc('X','V','I','D')

      # for horizontal flip augmentation
      if f==0:
        save_video_path=os.path.join('nprocessed_data/',video_dir,video.removeprefix(video_dirs+'/'))
      elif f==1:
        save_video_path=os.path.join('nprocessed_data/',video_dir,'f_'+video.removeprefix(video_dirs+'/'))

      videoWriter = cv2.VideoWriter(save_video_path, fourcc, 30.0, (224,224))

      with mp_pose.Pose(
          min_detection_confidence=0.5,
          min_tracking_confidence=0.5) as pose:
        with mp_hands.Hands(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as hands:
          while cap.isOpened():
            success, image = cap.read()
            if not success:
              print("Ignoring empty camera frame.")
              break
              # If loading a video, use 'break' instead of 'continue'.
              # continue

            image=cv2.resize(image,(224,224),interpolation=cv2.INTER_AREA)

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            results_p=pose.process(image)


            img=np.zeros_like(image,dtype=np.uint8)

            mp_drawing.draw_landmarks(
            img,
            results_p.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(),hands_or_pose='Pose')  

            if results.multi_hand_landmarks:
              for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    img,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing_styles.get_default_hand_landmarks_style(),
                    mp_drawing_styles.get_default_hand_connections_style(),
                    hands_or_pose='Hands')


            new_frame_time=time.time()
            fps = str(int(1/(new_frame_time-prev_frame_time)))
            prev_frame_time = new_frame_time

            # Flip the image horizontally for a selfie-view display.
            if f==1:
              # image=cv2.flip(image,1)
              img=cv2.flip(img,1)

            # save the mediapipe hand and pose video
            videoWriter.write(img)

            # cv2.putText(image, fps, (7, 70), cv2.FONT_HERSHEY_SIMPLEX , 3, (100, 255, 0), 3, cv2.LINE_AA)
            # cv2.imshow('MediaPipe Hand and Pose', img)      
            # cv2.imshow('Original',cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
            # if cv2.waitKey(5) & 0xFF == 27:
            #   break
      cap.release()
      videoWriter.release()
    cv2.destroyAllWindows()

## Real Time Stick-Figure Generation From Webcam Input

In [ ]:

import cv2
import mediapipe as mp
import numpy as np
import time
import os


mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands


prev_frame_time = 0
new_frame_time = 0


# For webcam input:
cap = cv2.VideoCapture('n3.mp4')
# cap = cv2.VideoCapture(0)
fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')


videoWriter = cv2.VideoWriter('slomo1.avi', fourcc, 30.0, (224,224))

with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
    with mp_hands.Hands(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                break
                # If loading a video, use 'break' instead of 'continue'.
                # continue

            image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = hands.process(image)
            results_p = pose.process(image)

            img = np.zeros_like(image, dtype=np.uint8)

            mp_drawing.draw_landmarks(
                img,
                results_p.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style(), hands_or_pose='Pose')

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(
                        img,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style(),
                        hands_or_pose='Hands')

            new_frame_time = time.time()
            fps = str(int(1/(new_frame_time-prev_frame_time)))
            prev_frame_time = new_frame_time

            # Flip the image horizontally for a selfie-view display.
            image = cv2.flip(image, 1)
            img = cv2.flip(img, 1)

            # save the mediapipe hand and pose video
            # videoWriter.write(img)

            cv2.putText(image, fps, (7, 70), cv2.FONT_HERSHEY_SIMPLEX,
                        3, (100, 255, 0), 3, cv2.LINE_AA)
            cv2.imshow('MediaPipe Hand and Pose', img)
            cv2.imshow('Original', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
            if cv2.waitKey(5) & 0xFF == 27:
                break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

## making the dataset

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path

In [ ]:
ds={'ne':['म घर मा धेरै काम गर्छु ।','मेरो घर भक्तपुर मा छ ।','म संग धेरै पैसा छैन ।','मेरो धेरै साथी हरु छन् ।','मेरो साथी हरु भक्तपुर मा छन् ।','तिम्रो काम धेरै छ ।',
          'म लाई अण्डा मनपर्छ ।','तिम्रो काम हरु म लाई छैन ।','म संग अण्डा छैन ।','तिम्रो काम छैन पैसा छैन ।','म तिम्रो पैसा खान्छु ।','तिमी संग अण्डा छैन ।',
          'तिमी म लाई मनपर्छ ।','भक्तपुर मा धेरै काम छ ।','म अण्डा खान्छु ।','तिमी हरु मेरो साथी हो ।','म भक्तपुर मा काम गर्छु ।','म लाई भक्तपुर मनपर्छ ।',
          'म संग मेरो साथी छ ।','मेरो साथी लाई अण्डा मनपर्छ ।']}
ds

In [ ]:
def get_all_sentences(ds,lang):
    for item in ds[lang]:
        yield item


def get_or_build_tokenizer(filename, ds, lang):
    tokenizer_path = Path(filename.format(lang))
    if not Path.exists(tokenizer_path):
        # Most code taken from: https://huggingface.co/docs/tokenizers/quicktour
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(
            special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=1)
        tokenizer.train_from_iterator(
            get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))

    return tokenizer

In [ ]:
# making the tokenizer

# tok=get_or_build_tokenizer('tokenizer_sign_lang_{0}.json',ds,'ne')

In [ ]:
tok.get_vocab()